In [1]:
import pandas as pd
import os

In [2]:
def load_csv(folder_path):
    dataframes = []
    file = 1

    for filename in os.listdir(path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            dataframes.append(df)
            print("File",file,"complete")
            file += 1

    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

path = "Datasets"
df = load_csv(path)
print(path, "loaded into one df.")

F:\WIN temp files\ipykernel_7780\924357319.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


File 1 complete


F:\WIN temp files\ipykernel_7780\924357319.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


File 2 complete
File 3 complete
File 4 complete
File 5 complete
File 6 complete
File 7 complete
File 8 complete
File 9 complete
File 10 complete
File 11 complete


F:\WIN temp files\ipykernel_7780\924357319.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


...

File 23 complete


F:\WIN temp files\ipykernel_7780\924357319.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


File 24 complete


F:\WIN temp files\ipykernel_7780\924357319.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


File 25 complete
Datasets


In [3]:
pickle_file_path = os.path.join(path, 'combined_data.pk1')
df.to_pickle(pickle_file_path)
print(f"DataFrame saved at: {pickle_file_path}")

Combined DataFrame saved as pickle file at: C:\Users\davev\PycharmProjects\FOE_Group_3\combined_data.pk1
